In [ ]:
# import sqlite3 and create a connection with a cursor
import sqlite3
connection = sqlite3.connect('student_grades.db')
cursor = connection.cursor()

In [ ]:
# Create students table
cursor.execute('''
CREATE TABLE IF NOT EXISTS students (
    student_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT,
    last_name TEXT
)
''')
# Create grades table
cursor.execute('''
CREATE TABLE IF NOT EXISTS grades (
    grade_id INTEGER PRIMARY KEY AUTOINCREMENT,
    student_id,
    subject TEXT,
    grade INTEGER,
    FOREIGN KEY (student_id) REFERENCES students(student_id)
)
''')


In [ ]:
# Create data for Students

student_data = [('Alice','Johnson'),
                ('Bob', 'Smith'),
                ('Carol','White'),
                ('David','Brown'),
                ('Eve','Davis')]

# Create data for Grades

grade_data = [(1,'Math',90),
              (1,'English',85),
              (1,'Science',95),
              (2,'Math',88),
              (2,'English',84),
              (2,'Science',98),
              (3,'Math',98),
              (3,'English',82),
              (3,'Science',99),
              (4,'Math',77),
              (4,'English',82),
              (4,'Science',80),
              (5,'Math',89),
              (5,'English',75),
              (5,'Science',98)]

# Empty Students and add data
cursor.execute("DELETE FROM students")
connection.commit()
cursor.executemany('''
INSERT INTO students (first_name,last_name) VALUES (?, ?)
''', student_data)
connection.commit()

# Empty Grades and add data
cursor.execute("DELETE FROM grades")
connection.commit()
cursor.executemany('''
INSERT INTO grades (student_id,subject,grade) VALUES (?, ?, ?)
''', grade_data)
connection.commit()

In [ ]:
# Retrieve all students names and grades
cursor.execute('''
SELECT students.first_name, students.last_name, grades.student_id, grades.subject, grades.grade
FROM students
JOIN grades ON students.student_id = grades.student_id
''')
joineddata = cursor.fetchall()

for row in joineddata:
    print(row)

In [ ]:
# Average grade by student
cursor.execute('''
SELECT students.first_name, students.last_name, AVG(grades.grade)
FROM students
JOIN grades ON students.student_id = grades.student_id
GROUP BY students.student_id
''')
avg_grades = cursor.fetchall()

for row in avg_grades:
    print(row)

In [ ]:
# Average grade by student
cursor.execute('''
SELECT students.first_name, students.last_name, AVG(grades.grade)
FROM students
JOIN grades ON students.student_id = grades.student_id
GROUP BY students.student_id
ORDER BY AVG(grades.grade) desc
''')
avg_grades = cursor.fetchone()

print(str(avg_grades[0]) + ' ' + str(avg_grades[1]) + ' has the highest grade average')

In [ ]:
# Math average
cursor.execute('''
SELECT grades.subject, AVG(grades.grade)
FROM grades
WHERE grades.subject = 'Math'
''')
math_avg = cursor.fetchone()

print(math_avg[1])

In [ ]:
# Distinct Individuals with a grade above 90
cursor.execute('''
SELECT DISTINCT students.first_name, students.last_name
FROM students
JOIN grades ON students.student_id = grades.student_id
WHERE grades.grade > 90
''')

above_90 = cursor.fetchall()

for row in above_90:
    print(row)

In [ ]:
# import pandas and get dataframe

import pandas as pd

students = pd.read_sql_query('SELECT * FROM students', connection)
grades = pd.read_sql_query('SELECT * FROM grades', connection)

In [ ]:
# join dataframes

joined_db = pd.merge(students,grades,how='inner',on='student_id')
joined_db.head()

In [ ]:
# import matplotlib
joined_db['name'] = joined_db['first_name'] + ' ' + joined_db['last_name']

mean_grades = joined_db[['name','grade']].groupby('name').mean()

import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
plt.bar(mean_grades.index,mean_grades['grade'])
plt.xlabel('Student')
plt.ylabel('Average Grade')
plt.title('Average Grade by Student')
plt.show()

mean_subjects = joined_db[['subject','grade']].groupby('subject').mean()

plt.bar(mean_subjects.index,mean_subjects['grade'])
plt.xlabel('Subject')
plt.ylabel('Average Grade')
plt.title('Average Grade by Subject')
plt.show()